In [ ]:
!pip install -U langchain-community
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00


In [ ]:
import openai

openai.api_key = "YOUR GPT API-KEY HERE"

try:
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt="Say hello!",
        max_tokens=5
    )
    print(response.choices[0].text.strip())
except openai.error.AuthenticationError as e:
    print("Authentication failed:", e)


AttributeError: module 'openai' has no attribute 'error'

In [ ]:
import openai
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import os

os.environ["OPENAI_API_KEY"] = ""  # Replace with your actual API key

documents = [
    "Generative AI is transforming industries like art, healthcare, and education.",
    "Multimodal AI combines text, image, and audio understanding into a single model.",
    "Transformers like GPT have revolutionized natural language processing."
]

document_objects = [Document(page_content=text) for text in documents]

text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(document_objects)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, embedding_model)

retriever = vector_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(model="gpt-3.5-turbo"),
    retriever=retriever,
    return_source_documents=True
)

print("Chatbot is ready! Ask your question:")
while True:
    query = input("\nYou: ")
    if query.lower() in {"exit", "quit"}:
        print("Exiting. Goodbye!")
        break

    try:
        response = qa_chain.invoke({"query": query})
        chatbot_answer = response["result"]
        print(f"\nChatbot: {chatbot_answer}")
        print("\n[Source Documents]:")
        for doc in response["source_documents"]:
            print(f"- {doc.page_content[:200]}...")  
    except Exception as e:
        print(f"An error occurred: {e}")


Chatbot is ready! Ask your question:

You: Hi
An error occurred: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


KeyboardInterrupt: Interrupted by user